<a href="https://colab.research.google.com/github/carlosbkm/ADaSci-Agentic-AI-Architect-labs/blob/main/Deploying_Agents_OpenAI_Agents_SDK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Library Installation

In [ ]:
!pip install pydantic-ai openai-agents nest-asyncio

### Patching asyncio for allowing nested use of asyncio.run

In [2]:
import nest_asyncio
nest_asyncio.apply()

### Library Imports and OpenAI_API_KEY Initialization

In [3]:
from agents import Agent, InputGuardrail,GuardrailFunctionOutput, Runner
from pydantic import BaseModel
from google.colab import userdata
import asyncio
import os

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

### Setting up Agents for Guardrails and Handoffs

In [4]:
class ResearchOutput(BaseModel):
    is_research: bool
    reasoning: str

guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking about research.",
    output_type=ResearchOutput,
)

cs_research_agent = Agent(
    name="CS Researcher",
    handoff_description="Specialist agent for Computer Science Research",
    instructions="You provide help with CS research. Explain your reasoning at each step and include examples",
)

bio_research_agent = Agent(
    name="Biology Researcher",
    handoff_description="Specialist agent for Biological Research",
    instructions="You assist with biological research. Explain important events and context clearly.",
)

### Defining Guardrail Trigger

In [5]:
async def research_guardrail(ctx, agent, input_data):
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(ResearchOutput)
    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=not final_output.is_research,
    )

### Handoff Setup

In [6]:
triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's research question",
    handoffs=[cs_research_agent, bio_research_agent],
    input_guardrails=[
        InputGuardrail(guardrail_function=research_guardrail),
    ],
)

### Explicit Tracer Setup

In [15]:
from agents.tracing.setup import GLOBAL_TRACE_PROVIDER
from agents import set_tracing_export_api_key

set_tracing_export_api_key(os.environ["OPENAI_API_KEY"])
GLOBAL_TRACE_PROVIDER._multi_processor.force_flush()

### Executing the Agent

In [16]:
# async def main():
#   user_input = "Explain the latest research on protein folding."
#   user_input_non_research = "Give me te recipe for a lemon cake"

#   try:
#     result = await Runner.run(triage_agent, user_input_non_research)
#     print(result.final_output)
#   except Exception as e:
#     print(f"Error: {e}")

#   result = await Runner.run(triage_agent, user_input)
#   print(result.final_output)
#     result = await Runner.run(triage_agent, "Explain the latest research on protein folding.")
#     print(result.final_output)

In [19]:
from agents import InputGuardrailTripwireTriggered

async def main():
    # Example 1: A query that is NOT about research (will trigger the error)
    non_research_query = "Give me te recipe for a lemon cake"

    print(f"--- Processing Input: '{non_research_query}' ---")

    # Use try...except to gracefully handle the guardrail trip
    try:
        # Attempt to run the agent
        result = await Runner.run(triage_agent, non_research_query)

        # This part runs ONLY if the guardrail DOESN'T trip
        print("✅ Guardrail Passed! Research query detected.")
        print(result.final_output)

    except InputGuardrailTripwireTriggered as e:
        # This block runs when the guardrail TRIPS (catches the error)
        print(f"⚠️ **GUARDRAIL TRIPWIRE ACTIVATED**")
        print(f"   Error caught: {e}")

        # ➡️ **The Solution:** Handle the request gracefully here.
        print("\n✨ **Graceful Action:** Redirecting or providing a fallback response...")
        print("   I'm sorry, I am currently specialized only for research-related queries.")
        # You would typically route the request to a different, more general agent here.

    print("\n------------------------------------------------------\n")

    # Example 2: A query that IS about research (will pass the guardrail)
    research_query = "Explain the latest research on protein folding."

    print(f"--- Processing Input: '{research_query}' ---")

    try:
          # Attempt to run the agent
          result = await Runner.run(triage_agent, research_query)

          # This part runs ONLY if the guardrail DOESN'T trip
          print("✅ Guardrail Passed! Research query detected.")
          print(result.final_output)

    except InputGuardrailTripwireTriggered as e:
          # This block will be skipped
          print(f"⚠️ Should not happen for this input: {e}")


if __name__ == "__main__":
  asyncio.run(main())

--- Processing Input: 'Give me te recipe for a lemon cake' ---
⚠️ **GUARDRAIL TRIPWIRE ACTIVATED**
   Error caught: Guardrail InputGuardrail triggered tripwire

✨ **Graceful Action:** Redirecting or providing a fallback response...
   I'm sorry, I am currently specialized only for research-related queries.

------------------------------------------------------

--- Processing Input: 'Explain the latest research on protein folding.' ---
✅ Guardrail Passed! Research query detected.
Certainly! Here’s a summary of the latest research and developments in the field of protein folding (as of mid-2024):

### 1. Deep Learning and Structure Prediction
- **AlphaFold 2 & Its Impact**: Since DeepMind’s release of AlphaFold 2 (2021), the field has been transformed. AlphaFold predicts the 3D structures of proteins from their amino acid sequences with remarkable accuracy, enabling many new discoveries in biology, medicine, and biotechnology. In 2023–24, further expansions (like AlphaFold-Multimer) al